# crypto

In [ ]:
open rust_operators

In [ ]:
//// test

open testing
open file_system_operators

## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"sha2::Sha256\")>]\n#endif\ntype sha2_Sha256 = class end"

In [ ]:
inl types () =
    sm'.types ()
    am'.types ()
    threading.types ()
    rust.types ()
    date_time.types ()
    file_system.types ()
    stream.types ()
    runtime.types ()
    types ()

## sha256

In [ ]:
nominal sha256 = $'System.Security.Cryptography.SHA256'

inl sha256 () : sha256 =
    $'`sha256.Create' ()

## sha256_compute_hash

In [ ]:
inl sha256_compute_hash (x : sha256) (data : a i32 u8) : a i32 u8 =
    data |> $'!x.ComputeHash'

## create_hash

In [ ]:
inl create_hash (x : string) : any =
    open ts_operators
    global "type ICryptoCreateHash = abstract createHash: x: string -> obj"
    inl crypto : $'ICryptoCreateHash' = ts.import_all "crypto"
    !\\(x, $'"!crypto.createHash($0)"')

## hash_update

In [ ]:
inl hash_update (s : string) (x : any) : any =
    open ts_operators
    !\\((x, s), $'"$0.update($1, \'utf8\')"')

## hash_digest

In [ ]:
inl hash_digest (s : string) (x : any) : string =
    open ts_operators
    !\\((x, s), $'"$0.digest($1)"')

## hash_text

In [ ]:
inl hash_text (input : string) =
    run_target function
        | Fsharp (Native) => fun () =>
            inl input = join input
            inl sha256 = sha256 () |> use
            input
            |> sm'.utf8_get_bytes
            |> sha256_compute_hash sha256
            |> am.map (sm'.byte_to_string "x2")
            |> seq.of_array'
            |> sm'.concat ""
        | TypeScript _ => fun () =>
            create_hash "sha256"
            |> hash_update input
            |> hash_digest "hex"
        | _ => fun () => null ()

In [ ]:
//// test

""
|> hash_text
|> _assert_eq "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"

assert_eq / actual: "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855" / expected: "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"


In [ ]:
//// test

" "
|> hash_text
|> _assert_eq "36a9e7f1c95b82ffb99743e0c5c4ce95d83c9a430aac59f84ef3cbfab6145068"

assert_eq / actual: "36a9e7f1c95b82ffb99743e0c5c4ce95d83c9a430aac59f84ef3cbfab6145068" / expected: "36a9e7f1c95b82ffb99743e0c5c4ce95d83c9a430aac59f84ef3cbfab6145068"


## get_file_hash'

In [ ]:
inl get_file_hash' (path : string) : result string string =
    inl path = path |> file_system.normalize_path
    inl exit_code, result =
        runtime.execution_options fun x => { x with
            command = $'$"pwsh -c \\\"(Get-FileHash \'{!path}\' -Algorithm SHA256).Hash\\\""'
        }
        |> runtime.execute_with_options
    if exit_code = 0
    then result |> sm'.to_lower |> Ok
    else result |> Error

In [ ]:
//// test

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()
disposable |> use |> ignore
inl file_name = "test.txt"
inl path = temp_folder </> file_name
"" |> file_system.write_all_text_async path |> async.run_synchronously
path
|> get_file_hash'
|> resultm.get
|> _assert_eq "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"

00:00:00   debug #1 execute_with_options_async / options: struct (None,
        "pwsh -c "(Get-FileHash 'c:/Users/i574n/AppData/Local/Temp/!dotnet-repl/20240519-0101-0602-0241-00040032da0f/test.txt' -Algorithm SHA256).Hash"",
        [||], None, None, true, None)
00:00:00 verbose #2 > E3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B934CA495991B7852B855
00:00:00   debug #3 execute_with_options_async / exit_code: 0 / output.Length: 64
assert_eq / actual: "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855" / expected: "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"


In [ ]:
//// test
///! rust -d chrono encoding_rs encoding_rs_io futures futures-lite regex

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()
inl file_name = "test.txt"
inl path = temp_folder </> file_name
"" |> file_system.write_all_text path
path
|> get_file_hash'
|> resultm.get
|> _assert_eq "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"
disposable |> use |> ignore

00:00:00 verbose #1 file_system.create_directory / dir: C:\Users\i574n\AppData\Local\Temp\!spiral_builder_28e239ae45fa411d5446d41d1a99f210110076943882edefe0706e85dc5a94e5\20240519-0101-4292-1658-000000857575
00:00:00   debug #2 runtime.execute_with_options / file_name: pwsh / arguments: ["-c", "(Get-FileHash 'c:/Users/i574n/AppData/Local/Temp/!spiral_builder_28e239ae45fa411d5446d41d1a99f210110076943882edefe0706e85dc5a94e5/20240519-0101-4292-1658-000000857575/test.txt' -Algorithm SHA256).Hash"] / options: (None, "pwsh -c "(Get-FileHash 'c:/Users/i574n/AppData/Local/Temp/!spiral_builder_28e239ae45fa411d5446d41d1a99f210110076943882edefe0706e85dc5a94e5/20240519-0101-4292-1658-000000857575/test.txt' -Algorithm SHA256).Hash"", Array(MutCell([])), None, None, true, None)
00:00:00 verbose #3 > E3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B934CA495991B7852B855
00:00:00 verbose #4 runtime.execute_with_options / result / exit_code: 0 / std_trace.Length: 64
assert_eq / actual: "e3b0c44298fc1c149afbf

## sha256'

In [ ]:
nominal sha256' = $'sha2_Sha256'

## new_sha256

In [ ]:
inl new_sha256 () : sha256' =
    !\($'"sha2::Digest::new()"')

## hasher_update

In [ ]:
inl hasher_update forall el dim. (slice : rust.ref' (am'.slice' el dim)) (hasher : sha256') : () =
    !\($'"sha2::Digest::update(&mut !hasher, !slice)"')

## hasher_finalize

In [ ]:
inl hasher_finalize (hasher : sha256') : rust.ref' (am'.slice u8) =
    !\($'"&sha2::Digest::finalize(!hasher)"')

## get_file_hash

In [ ]:
inl get_file_hash (path : string) : result string string =
    inl path = path |> file_system.normalize_path
    inl file = path |> file_system.file_open |> resultm.unwrap'
    inl reader = file |> stream.new_buf_reader
    (!\($'"true; let mut !reader = !reader"') : bool) |> ignore
    inl hasher = new_sha256 ()
    (!\($'"true; let mut !hasher = !hasher"') : bool) |> ignore
    
    real
        inl size = 1024
        inl zero = unativeint `i32 0
        inl buffer = am'.new_slice `u8 `@size 0u8

        rust.loop 2 fun () =>
            inl count = stream.buf_reader_read `u8 `@size buffer reader
            inl count = resultm.unwrap' `unativeint `(stream.io_error) count

            if (=.) `unativeint count zero then rust.break ()

            hasher_update `u8 `@size
                (
                    am'.slice_range `u8 `@size
                        (am'.Start `unativeint zero)
                        (am'.End `unativeint ((fun _ => count) : unativeint -> unativeint))
                        buffer
                )
                hasher

    hasher
    |> hasher_finalize
    |> am'.slice_to_vec
    |> am'.vec_map' (sm'.format_custom' "{:02x}" >> sm'.from_std_string)
    |> am'.from_vec
    |> fun x => x : _ i32 _
    |> seq.of_array'
    |> sm'.concat ""
    |> Ok

In [ ]:
//// test
///! rust -d chrono regex sha2

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()

inl file_name = join "test.txt"
inl path = temp_folder </> file_name
"" |> file_system.write_all_text path
path
|> get_file_hash
|> resultm.get
|> _assert_eq "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"
disposable |> use |> ignore

00:00:00 verbose #1 file_system.create_directory / dir: C:\Users\i574n\AppData\Local\Temp\!spiral_builder_f2f837a3fd3f31a551997578d454aa64c2d89813b04e431485cd82153c8f5329\20240519-0101-5751-9589-0000009935ac
assert_eq / actual: "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855" / expected: "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"


In [ ]:
//// test
///! rust -d chrono regex sha2

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()

inl file_name = join "test.txt"
inl path = temp_folder </> file_name
" " |> file_system.write_all_text path
path
|> get_file_hash
|> resultm.get
|> _assert_eq "36a9e7f1c95b82ffb99743e0c5c4ce95d83c9a430aac59f84ef3cbfab6145068"
disposable |> use |> ignore

00:00:00 verbose #1 file_system.create_directory / dir: C:\Users\i574n\AppData\Local\Temp\!spiral_builder_10eb4cf90f035e50d5826a64b2742e7c11f22d4d8acd61507eb4adb594e3e141\20240519-0102-0519-7299-0000007fc3cd
assert_eq / actual: "36a9e7f1c95b82ffb99743e0c5c4ce95d83c9a430aac59f84ef3cbfab6145068" / expected: "36a9e7f1c95b82ffb99743e0c5c4ce95d83c9a430aac59f84ef3cbfab6145068"


## main

In [ ]:
inl main () =
    types ()
    $"let hash_text x = !hash_text x" : ()